In [ ]:
#importo
#mustro datos
#elimino columnas que no necesito
#Elimino status que no necesito
#Elimino los nulos
#Cambio charge off y dafault por No Paid
#Outliders
#One hot encoder de: purpose, 


In [1]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
import itertools
import warnings
import plotly.express as px
from sklearn.preprocessing import minmax_scale
warnings.filterwarnings("ignore")

In [2]:
#Importar csv
original_data=pd.read_csv('../dataset/loans_data.csv')

In [3]:
#Datos originales
original_data.head()

,funded_amnt,term,int_rate,installment,emp_title,emp_length,home_ownership,annual_inc,issue_d,loan_status,...,open_acc,total_pymnt,last_pymnt_amnt,acc_now_delinq,tot_coll_amt,delinq_amnt,disbursement_method,total_bal_ex_mort,grade,dti
0,2500,36 months,13.56,84.92,Chef,10+ years,RENT,55000.0,Dec-2018,Current,...,9.0,167.02,84.92,0.0,0.0,0.0,Cash,16901.0,C,18.24
1,30000,60 months,18.94,777.23,Postmaster,10+ years,MORTGAGE,90000.0,Dec-2018,Current,...,13.0,1507.11,777.23,0.0,1208.0,0.0,Cash,99468.0,D,26.52
2,5000,36 months,17.97,180.69,Administrative,6 years,MORTGAGE,59280.0,Dec-2018,Current,...,8.0,353.89,180.69,0.0,0.0,0.0,Cash,11749.0,D,10.51
3,4000,36 months,18.94,146.51,IT Supervisor,10+ years,MORTGAGE,92000.0,Dec-2018,Current,...,10.0,286.71,146.51,0.0,686.0,0.0,Cash,36151.0,D,16.74
4,30000,60 months,16.14,731.78,Mechanic,10+ years,MORTGAGE,57250.0,Dec-2018,Current,...,12.0,1423.21,731.78,0.0,0.0,0.0,Cash,29674.0,C,26.35


In [4]:
#Quitar status de loan que no nos interesan

new_data=original_data.drop(original_data[original_data['loan_status']=='Current'].index)

new_data=new_data.drop(new_data[new_data['loan_status']=='Late (16-30 days)'].index)
new_data=new_data.drop(new_data[new_data['loan_status']=='Late (31-120 days)'].index)
new_data=new_data.drop(new_data[new_data['loan_status']=='In Grace Period'].index)
new_data=new_data.drop(new_data[new_data['loan_status']=='Does not meet the credit policy. Status:Fully Paid'].index)
new_data=new_data.drop(new_data[new_data['loan_status']=='Does not meet the credit policy. Status:Charged Off'].index)


In [5]:
#Quita columnas
new_data=new_data.drop(['emp_title','home_ownership','title','disbursement_method','earliest_cr_line','issue_d','last_pymnt_amnt','open_acc','total_bal_ex_mort','tot_coll_amt'],axis=1)

In [6]:
#Total de nulos
new_data.isnull().sum()

funded_amnt           0
term                  0
int_rate              0
installment           0
emp_length        75457
annual_inc            0
loan_status           0
purpose               0
total_pymnt           0
acc_now_delinq        0
delinq_amnt           0
grade                 0
dti                 312
dtype: int64

In [7]:
#Quita nulos
new_data= new_data.dropna()

In [8]:
#volver Target Binario
new_data=new_data.replace(regex=['Default','Charged Off'],value='No paid')

In [9]:
#Se cree nuevo dataset 'process_data' con los datos procesados
process_data=new_data.copy()

In [10]:
#Escalar:funded_amnt,int_rate,installment,annual_inc,total_pymnt,total_coll_amnt,delinq_amnt,dti
process_data[['funded_amnt','int_rate','installment','annual_inc','total_pymnt','delinq_amnt','dti']]= minmax_scale(new_data[['funded_amnt','int_rate','installment','annual_inc','total_pymnt','delinq_amnt','dti']])

In [11]:
new_data.purpose.unique()

array(['debt_consolidation', 'credit_card', 'other', 'house', 'car',
       'home_improvement', 'moving', 'small_business', 'medical',
       'major_purchase', 'renewable_energy', 'vacation', 'wedding',
       'educational'], dtype=object)

In [12]:
process_data.purpose.unique()

array(['debt_consolidation', 'credit_card', 'other', 'house', 'car',
       'home_improvement', 'moving', 'small_business', 'medical',
       'major_purchase', 'renewable_energy', 'vacation', 'wedding',
       'educational'], dtype=object)

In [13]:
original_data.purpose.unique()

array(['debt_consolidation', 'credit_card', 'house', 'car', 'other',
       'vacation', 'home_improvement', 'small_business', 'major_purchase',
       'medical', 'renewable_energy', 'moving', 'wedding', 'educational'],
      dtype=object)

In [14]:
#One Hot-Ecoder Variable: Purpose

for purpose in process_data.purpose.unique():
    process_data[purpose.lower().replace(" ","_")]= (process_data.purpose==purpose)

process_data=process_data.drop(['purpose'],axis=1)



In [15]:
pd.set_option('display.max_columns',None)
process_data.head()


,funded_amnt,term,int_rate,installment,emp_length,annual_inc,loan_status,total_pymnt,acc_now_delinq,delinq_amnt,grade,dti,debt_consolidation,credit_card,other,house,car,home_improvement,moving,small_business,medical,major_purchase,renewable_energy,vacation,wedding,educational
100,0.746835,36 months,0.663551,0.668395,5 years,0.009089,Fully Paid,0.474375,0.0,0.0,D,0.03146,True,False,False,False,False,False,False,False,False,False,False,False,False,False
152,1.000000,60 months,0.421729,0.566085,< 1 year,0.004088,Fully Paid,0.645477,0.0,0.0,C,0.05153,False,True,False,False,False,False,False,False,False,False,False,False,False,False
170,0.493671,36 months,0.087617,0.360225,10+ years,0.009089,Fully Paid,0.319381,0.0,0.0,A,0.01992,False,True,False,False,False,False,False,False,False,False,False,False,False,False
186,0.101266,36 months,0.233645,0.083422,10+ years,0.003497,Fully Paid,0.071871,0.0,0.0,B,0.00564,False,True,False,False,False,False,False,False,False,False,False,False,False,False
215,0.200633,36 months,0.855140,0.198408,3 years,0.040909,Fully Paid,0.138003,0.0,0.0,E,0.01337,False,True,False,False,False,False,False,False,False,False,False,False,False,False


In [27]:
#Datos a utilizar
new_data.head()

,funded_amnt,term,int_rate,installment,emp_length,annual_inc,loan_status,purpose,total_pymnt,acc_now_delinq,delinq_amnt,grade,dti
100,30000,36 months,22.35,1151.16,5 years,100000.0,Fully Paid,debt_consolidation,30026.442500,0.0,0.0,D,30.46
152,40000,60 months,16.14,975.71,< 1 year,45000.0,Fully Paid,credit_card,40856.678960,0.0,0.0,C,50.53
170,20000,36 months,7.56,622.68,10+ years,100000.0,Fully Paid,credit_card,20215.792430,0.0,0.0,A,18.92
186,4500,36 months,11.31,147.99,10+ years,38500.0,Fully Paid,credit_card,4549.217149,0.0,0.0,B,4.64
215,8425,36 months,27.27,345.18,3 years,450000.0,Fully Paid,credit_card,8735.148975,0.0,0.0,E,12.37


In [ ]:
#Tipo de datos de cada columna
new_data.dtypes

In [ ]:
#Datos de salida Balanceo de datos

import matplotlib.pyplot as plt
import numpy as np

x=new_data.loan_status.unique()
y=new_data.loan_status.value_counts()

colors = ['yellowgreen','blue','gold','lightskyblue','coral','grey','pink', 'green','violet']
porcent = 100.*y/y.sum()

patches, texts = plt.pie(y, colors=colors, startangle=90, radius=3)
labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(x, porcent)]

sort_legend = True
if sort_legend:
    patches, labels, dummy =  zip(*sorted(zip(patches, labels, y),
                                          key=lambda x: x[2],
                                          reverse=True))

plt.legend(patches, labels, loc='left center', bbox_to_anchor=(-0.1, 1.),
           fontsize=15)

plt.savefig('piechart.png', bbox_inches='tight')

In [ ]:
#VARIABLE GRADE
px.histogram(new_data.sort_index(),x='grade',color='loan_status',barmode='group')


In [ ]:
#VARIABLE PURPOSE (CON HOT-ENCODER)

px.histogram(new_data.sort_index(),x='puurpose',color='loan_status',barmode='group')

In [ ]:
px.histogram(new_data.sort_index(),x='grade',color='loan_status',barmode='group')

In [ ]:
#VARIABLE ANNUAL_INC
#Crea DataFrame para obtener rangos de ingresos anuales y poder graficar
range_inc=pd.DataFrame()
range_inc['loan_status']=new_data['loan_status']
range_inc['annual_inc']=new_data['annual_inc']
range_inc['ranges_inc']=pd.cut(x=range_inc['annual_inc'],bins=[0,25000,50000,75000,100000,125000,150000,1000000],labels=['Entre 0 y 25k','Entre 25k y 50k','Entre 50k y 75k','Entre 75k y 100k','Entre 100k y 125k','Entre 125k y 150k','Mayor a 150k'])


px.histogram(range_inc,x='ranges_inc',color='loan_status',barmode='group')




In [ ]:
#VARIABLE FUNDED_AMNT
#Crea DataFrame para obtener rangos de ingresos anuales y poder graficar
range_funded_amnt=pd.DataFrame()
range_funded_amnt['loan_status']=new_data['loan_status']
range_funded_amnt['funded_amnt']=new_data['funded_amnt']
range_funded_amnt['ranges_amnt']=pd.cut(x=range_funded_amnt['funded_amnt'],bins=[5000,10000,15000,20000,25000,30000,35000,40000],labels=['Menor a 5k','Entre 5k y 10k','Entre 10k y 15k','Entre 15k y 20k','Entre 20k y 25k','Entre 25k y 30k','Mayor a 35k'])


px.histogram(range_funded_amnt,x='ranges_amnt',color='loan_status',barmode='group')

In [ ]:
#VARIABLE EMP_LENGTH
px.histogram(new_data,x='emp_length',color='loan_status',barmode='group')